# 오늘은 LeNet 구조를 만들어봅시다


LeNet 구조는 CNN이며, 초기에 만들어진 모델입니다. 

2가지 모델(Sigmoid, ReLU)를 만들어 두 모델의 성능을 비교해봅시다.


## 1.우선 필요 라이브러리를 import 합니다.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torchvision import datasets
import torchvision.transforms as transforms

import torch.optim as optim

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

## 2. 딥러닝 모델을 설계할 때 활용하는 장비 확인

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('Using PyTorch version:', torch.__version__, ' Device:', device)

Using PyTorch version: 1.11.0  Device: cpu


## 3. MNIST 데이터 다운로드 

 1. Training data와 Test data 분리하기
 
 2. Training data를 Training data 와 Validation data로 분리하기

In [3]:
BATCH_SIZE = 64


transform = transforms.Compose(
    [
     transforms.ToTensor(),                    # 데이터를 Tensor 형태로 변형 (PIL Image / numpy.ndarray 를 Tesnor 로 변형)
     transforms.Normalize(mean = (0.1307,), std = (0.3081,)),     # 데이터 정규화 (Grayscale 이므로, Channel 은 1)
    ])

train_data = datasets.MNIST('./MNIST_DATASET',train=True,download=True,transform=transform)  # MNIST 데이터 셋 다운로드/변형
test_data = datasets.MNIST('./MNIST_DATASET',train=False,download=True,transform=transform)        #채우세요
##--------------------------------------------------------------------------------------------------------------

train, val = torch.utils.data.random_split(train_data,[int(0.8*len(train_data)), int(0.2*len(train_data))])   #8:2로 분리
                                                                         

train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=BATCH_SIZE,shuffle=False,drop_last=True) 
val_loader = torch.utils.data.DataLoader(dataset=val, batch_size=BATCH_SIZE,shuffle=False,drop_last=True)     
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=BATCH_SIZE,shuffle=False,drop_last=True)  


## 4. torch.nn을 이용하여 모델-1 만들기

   1) 아래의 그림 중 LeNet 구조를 구현 할 것
   
   2) Sigmoid 활성화 함수를 이용할 것
   
   
![](Comparison_image_neural_networks.svg.png)

In [4]:
import torch.nn as nn

class Model_1(nn.Module):                  # Pytorch 에서의 신경망 정의 (torch.nn.Module 을 반드시 상속)
    
    def __init__(self):                  
        super(Model_1, self).__init__()    
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, padding=2, kernel_size=5, stride=1) 
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1)   
        self.fc1 = nn.Linear(16*5*5,120)   # 은닉층 (완전연결 층)으로,120 개의 Neuron 존재, 이때, Input 으로는 직전 Pooling 의 Output을 선형화한 16x5x5 개의 데이터들
        self.fc2 = nn.Linear(120, 84)      # 은닉층 (완전연결 층)으로,84 개의 Neuron 존재 
        self.fc3 = nn.Linear(84, 10)       # 은닉층 (완전연결 층)으로,10 개의 Neuron 존재
        
    def forward(self,x):                   # 순전파 함수 Overriding 
        x = nn.functional.max_pool2d(torch.sigmoid(self.conv1(x)), (2, 2)) # Convolution Layer 1의 Output 을 Maxpooling 계층으로 연결
        x = nn.functional.max_pool2d(torch.sigmoid(self.conv2(x)), (2, 2))  # Convolution Layer 2의 Output 을 Maxpooling 계층으로 연결
        #print(x.shape())
        x = x.view(-1,5*5*16)                                   # 다음, 완전연결 층에서, 행렬 곱 연산이 가능토록, Input Reshape
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        x = self.fc3(x)
        return x
                                           


## 5. torch.nn을 이용하여 모델-2 만들기

   LeNet 모델에서 ReLU 활성화 함수를 사용하시요

In [5]:
class Model_2(nn.Module):
    def __init__(self):
        super(Model_2, self).__init__()    
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, padding=2, kernel_size=5, stride=1)  
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1)           
        self.fc1 = nn.Linear(16*5*5,120)   # 은닉층 (완전연결 층)으로,120 개의 Neuron 존재, 이때, Input 으로는 직전 Pooling 의 Output을 선형화한 16x5x5 개의 데이터들
        self.fc2 = nn.Linear(120, 84)      # 은닉층 (완전연결 층)으로,84 개의 Neuron 존재 
        self.fc3 = nn.Linear(84, 10)       # 은닉층 (완전연결 층)으로,10 개의 Neuron 존재
        
    def forward(self,x):
        x = nn.functional.max_pool2d(nn.functional.relu(self.conv1(x)), (2, 2)) # Convolution Layer 1의 Output 을 Maxpooling 계층으로 연결
        x = nn.functional.max_pool2d(nn.functional.relu(self.conv2(x)), (2, 2))  # Convolution Layer 2의 Output 을 Maxpooling 계층으로 연결
        x = x.view(-1,5*5*16)                                   # 다음, 완전연결 층에서, 행렬 곱 연산이 가능토록, Input Reshape
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x
        
        return x

## 7. 학습 준비하기

1) 1 epoch를 학습할 수 있는 함수 만들기

2) Test와 Validation data의 정확도 계산할 수 있는 함수 만들기

In [6]:
def training_epoch(train_loader, network, loss_func, optimizer, epoch): # 1 Epoch 학습 할 수 있는 함수
    '''
    trainer_loader : Training Sample 에 대한 DataLoader
    network : 앞서 제작한 LeNET 신경망 객체
    loss_func : Loss 계산에 사용할 nn.functional 소속 메소드 (Loss 함수)
    optimizer :
    epoch : 
    '''
    
    train_losses = []                     # 현재 학습 단게에서의 Loss 값
    train_correct = 0                     # 학습 정확도 0으로 초기화
    log_interval = 300                    # 300 개의 Batch 단위로 Loss 출력
    
    for batch_idx, (image, label) in enumerate(train_loader):
        '''
        batch_idx : Training Sample 에 포함된 N 개의 Batch 들 중 몇 번째 Batch 인가?
        image : i 번째 Batch 에 대한 Input Data
        label : Input Data 에 대한 정답 Label
        '''
        image, label = image.to(device), label.to(device)

        # 미분값의 초기화
        optimizer.zero_grad() 
        # 미분을 통해 얻은 기울기를 0으로 초기화함. 기울기를 초기화해야만 새로운 가중치 편향에 대해서 새로운 기울기를 구할 수 있음

        # Forward propagration 계산하기.
        outputs = network.forward(image)
        # Input Data 를 이용한 순전파 실시
        
        
        # Cross_entropy 함수를 적용하여 loss를 구하고 저장하기
        loss = loss_func(outputs,label)       # 최종 결과값과 Label 값 비교하여 Loss 계산 (Cross Entropy 계산)         
        train_losses.append(loss.item())                # 각 Input Data들에 대한 Loss 계산하여, Append

        # training accuracy 정확도 구하기 위해 맞는 샘플 개수 세기
        pred = torch.argmax(outputs.data, 1)            
        train_correct += pred.eq(label).sum()           # pred.eq(label) : pred, label 과 Element-Wise Level 동등 비교
        # Gradinet 구하기
        loss.backward()  

        # weight값 update 하기
        optimizer.step()                # 인수로 들어갔던 W와 b에서 리턴되는 변수들의 기울기에 학습률(learining rate) 0.01을 곱하여 빼줌으로서 업데이트

        # 학습 상황 출력
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.2f}%)]\tLoss: {:.6f}'
                  .format(epoch, batch_idx * len(label), len(train_loader.dataset),100. * batch_idx / len(train_loader),
                          loss.item()))
            
    return train_losses, train_correct

In [7]:
def test_epoch(test_loader, network, loss_func, val = False): # 1 회 Test 할 수 있는 함수
    '''
    test_loader : Training Sample 에 대한 DataLoader
    network : 앞서 제작한 LeNET 신경망 객체
    loss_func : Loss 계산에 사용할 nn.functional 소속 메소드 (Loss 함수)
    val : Validation 데이터에 대한 Testing (True), Testing 데이터에 대한 Testing (False)
    '''
    correct = 0        # Test 정확도 0으로 초기화
    
    test_losses = [] 
    
    with torch.no_grad():
        for batch_idx, (image, label) in enumerate(test_loader):          # Test Sample 에 포함된 각각의 Batch 들 마다 테스트 진행
            
            image, label = image.to(device), label.to(device)

            # Forward propagration 계산하기.
            outputs = network.forward(image)

            # Cross_entropy 함수를 적용하여 loss를 구하기
            loss = loss_func(outputs,label) 
            test_losses.append(loss.item())

            # Batch 별로 정확도 구하기
            pred = torch.argmax(outputs.data, 1)
            correct += pred.eq(label).sum()

        # 전체 정확도 구하기
        test_accuracy = 100. * correct / len(test_loader.dataset)         # 백분율로 환산할 것!

        
        if val is True:                                                  # Validation 데이터에 대한 정확도 출력 (백분율로!)
                print('Validation set: Accuracy: {}/{} ({:.2f}%)\n'
              .format(correct, len(test_loader.dataset),100. * correct / len(test_loader.dataset)))
        
        else:
            print('Test set: Accuracy: {}/{} ({:.2f}%)\n'                # Test 데이터에 대한 정확도 출력 (백분율로!)
                  .format(correct, len(test_loader.dataset),100. * correct / len(test_loader.dataset)))
        
    return test_losses, test_accuracy


## 8. 위 정의된 함수로 학습 함수 만들기

Adam Optimizer를 사용하여 학습시키기

In [8]:
def training(network, learning_rate = 0.001):
    
    epoches = 15                  # 전체 50000 개의 Data Sample 들에 대한 학습을 1 Epoch 라 가정,총 15 Ecpoch 학습 실시
    
    cls_loss = nn.CrossEntropyLoss()                                                # 손실 함수로 Cross Entropy 사용
    optimizer = optim.Adam(network.parameters(), lr=learning_rate, weight_decay=0.1) 
    
    '''
    SGD 학습 사용 (전체 50000//64 Batch 에서 임의의 하나의 Batch 선택하여 학습)
    학습률 (0.001) 사용
    가중치 감소 비율 (0.1) 사용
    '''
    
    train_losses_per_epoch = []                                 # 1 Epoch 학습 당 손실
    test_losses_per_epoch = []                                  # 1 Epoch Test 당 손실   
    
    train_accuracies = []
    test_accuracies = []
    
    
    for epoch in range(epoches):                               # 15 Epoch 만큼 학습 실시
                
        # 모델를 학습 중이라고 선언하기
        network.train()                                       
        
        train_losses, train_correct = training_epoch(train_loader,network,cls_loss,optimizer, epoch)
        
        # epoch 별로 loss 평균값, 정확도 구하기
        average_loss = np.mean(train_losses)
        train_losses_per_epoch.append(average_loss)
        
        train_accuracy = train_correct / len(train_loader.dataset) * 100
        train_accuracies.append(train_accuracy)
        
        # epoch 별로 정확도 출력
        print('\nTraining set: Accuracy: {}/{} ({:.2f}%)'
              .format(train_correct, len(train_loader.dataset),100. * train_correct / len(train_loader.dataset)))

        
        ### 학습 중에 test 결과 보기
        
        # 모델 test 중인 것을 선언하기
        network.eval()
        
        correct = 0
        with torch.no_grad():
            test_losses, test_accuracy = test_epoch(val_loader, network, cls_loss, True)

        test_losses_per_epoch.append(np.mean(test_losses))
        test_accuracies.append(test_accuracy)
        
    with torch.no_grad():
        test_losses, test_accuracy = test_epoch(test_loader, network, cls_loss, False)
        
    return train_losses_per_epoch, test_losses_per_epoch, train_accuracies, test_accuracies


In [9]:
network = Model_1().to(device)
rlt_const = training(network)

Train Epoch: 0 [0/48000 (0.00%)]	Loss: 2.397290
Train Epoch: 0 [19200/48000 (40.00%)]	Loss: 2.323895
Train Epoch: 0 [38400/48000 (80.00%)]	Loss: 2.260111

Training set: Accuracy: 4925/48000 (10.26%)
Validation set: Accuracy: 1185/12000 (9.88%)

Train Epoch: 1 [0/48000 (0.00%)]	Loss: 2.323536
Train Epoch: 1 [19200/48000 (40.00%)]	Loss: 2.324300
Train Epoch: 1 [38400/48000 (80.00%)]	Loss: 2.260082

Training set: Accuracy: 4919/48000 (10.25%)
Validation set: Accuracy: 1185/12000 (9.88%)

Train Epoch: 2 [0/48000 (0.00%)]	Loss: 2.323750
Train Epoch: 2 [19200/48000 (40.00%)]	Loss: 2.324445
Train Epoch: 2 [38400/48000 (80.00%)]	Loss: 2.260014

Training set: Accuracy: 4920/48000 (10.25%)
Validation set: Accuracy: 1185/12000 (9.88%)

Train Epoch: 3 [0/48000 (0.00%)]	Loss: 2.323849
Train Epoch: 3 [19200/48000 (40.00%)]	Loss: 2.324513
Train Epoch: 3 [38400/48000 (80.00%)]	Loss: 2.260004

Training set: Accuracy: 4917/48000 (10.24%)
Validation set: Accuracy: 1185/12000 (9.88%)

Train Epoch: 4 [0/48

In [10]:
network = Model_2().to(device)
rlt_const = training(network)

Train Epoch: 0 [0/48000 (0.00%)]	Loss: 2.324073
Train Epoch: 0 [19200/48000 (40.00%)]	Loss: 0.844715
Train Epoch: 0 [38400/48000 (80.00%)]	Loss: 0.400623

Training set: Accuracy: 34568/48000 (72.02%)
Validation set: Accuracy: 10402/12000 (86.68%)

Train Epoch: 1 [0/48000 (0.00%)]	Loss: 0.435991
Train Epoch: 1 [19200/48000 (40.00%)]	Loss: 0.471510
Train Epoch: 1 [38400/48000 (80.00%)]	Loss: 0.345997

Training set: Accuracy: 43135/48000 (89.86%)
Validation set: Accuracy: 10679/12000 (88.99%)

Train Epoch: 2 [0/48000 (0.00%)]	Loss: 0.362746
Train Epoch: 2 [19200/48000 (40.00%)]	Loss: 0.375910
Train Epoch: 2 [38400/48000 (80.00%)]	Loss: 0.328153

Training set: Accuracy: 43863/48000 (91.38%)
Validation set: Accuracy: 10905/12000 (90.88%)

Train Epoch: 3 [0/48000 (0.00%)]	Loss: 0.324384
Train Epoch: 3 [19200/48000 (40.00%)]	Loss: 0.367007
Train Epoch: 3 [38400/48000 (80.00%)]	Loss: 0.319711

Training set: Accuracy: 44065/48000 (91.80%)
Validation set: Accuracy: 10987/12000 (91.56%)

Train Ep

## 9. 두모델의 성능을 비교하시오

정답)
활성화 함수로 ReLu를 사용했을 때가 Sigmoid를 사용했을 때보다 월등한 성능을 가진다. 
ReLu를 사용했을 때는 epoch가 지남에 따라 학습이 제대로 되는 반면, Sigmoid는 학습이 제대로 되지 않았다.
이는 활성화 함수의 특성 때문이다. ReLU의 수렴 속도는 Tanh 같은 함수보다 훨씬 빠르다. 또한 활성화 값을 구하기 위해서는 임계값(threshold  value)만 필요하다. 즉 다른 활성화 함수에 비해 복잡한 계산이 없고 gradient descent시에 시간을 절약할 수 있다. ReLU 기능은 NN 희소성을 초래하는 일부 뉴런의 출력을 0으로 만들고 매개 변수의 상호의존성을 감소시키며 과적합 문제를 완화하고 전체적으로 더 효율적으로 만들 수 있다.
반면에 Sigmoid는 출력 값이 극한에 가까워 질수록 기존 S자 모양이던 곡선이 점점 수평이 되어 간다. 이것은 출력 값들을 오직 0,1에 가까운 값으로만 압축을 하는 'saturation' 현상을 초래한다. 그렇기에 입력 값이 너무 클 경우 data loss를 일으켜 학습이 진행되지 않는다.

참고자료 : https://iopscience.iop.org/article/10.1088/1755-1315/428/1/012097